# **The Dataset**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import KBinsDiscretizer


## The dataset is sourced from Kaggle and imported into my personal Github Repository
## The dataset contains COVID-19 information related to cases, deaths, recoveries, etc.
url = 'https://raw.githubusercontent.com/AndrewDucDanhDo/machine_learning_assessment_2_dataset/main/country_wise_latest.csv'
raw_data = pd.read_csv(url)


attributes = ['Confirmed','Deaths', 'Recovered', 'Active', 'New cases', 'New deaths', 'New recovered']
target = "Habitability"

# Divide the dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(raw_data[attributes], raw_data[target], test_size=0.3, shuffle=False)

# Robust Scaler will transform the dataset in consideration of the extreme outliers (Ex. US COVID-19 data is highest in the world)
X_train_robust_scaler = RobustScaler(quantile_range=(25, 75)).fit_transform(X_train)
X_test_robust_scaler = RobustScaler(quantile_range=(25, 75)).fit_transform(X_test)

# The minmax_scale scales the dataset between 0-1
X_train_scale = minmax_scale(X_train_robust_scaler)
X_test_scale = minmax_scale(X_test_robust_scaler)


# The dataset is then binned so that the data is categorical instead of continuous
enc = KBinsDiscretizer(n_bins=6, encode='ordinal')
X_train_binned = enc.fit_transform(X_train_scale)

enc = KBinsDiscretizer(n_bins=6, encode='ordinal')
X_test_binned = enc.fit_transform(X_test_scale)

# Convert the dataset back to a csv file for later usage
X_train_processed = pd.DataFrame(X_train_binned, columns=[attributes])
X_train_processed.to_csv('i.csv', index=False, header=False)

X_test_processed = pd.DataFrame(X_test_binned, columns=[attributes])
X_test_processed.to_csv('i.csv', index=False, header=False)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 5 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 6 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 5 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.

In [ ]:
# Decides whether the binned dataset is Habitable for the training dataset to build the tree and for later comparison with the test dataset (answers)
def check_habitable(dataset):
    l = []
    for i in  range(len(dataset)):
        x = 0
        for j in range(len(dataset[0])):       
            if ((i == 0 or i == 1 or i == 3) and dataset[i][j]>3):
                x +=1
            elif (i == 2 and dataset[i][j] <= 3):
                x += 1
            else:
                continue

        if (x > 2):
            l.append("No")
        else:
            l.append("Yes")
    
    return l

In [ ]:
## Train "Habitabilty" column dataset used to build the decision tree
train_habitable = check_habitable(X_train_binned)
y_train_processed = pd.DataFrame(train_habitable, columns=['Habitability'])
y_train_processed.to_csv('i.csv', index=False, header=False)

## Test "Habitability" column dataset used for answer comparison with the predicted
test_habitable = check_habitable(X_test_binned)
y_test_processed = pd.DataFrame(test_habitable, columns=['Habitability'])
y_test_processed.to_csv('i.csv', index=False, header=False)

print(y_test_processed)

   Habitability
0           Yes
1           Yes
2           Yes
3            No
4           Yes
5           Yes
6           Yes
7           Yes
8           Yes
9           Yes
10          Yes
11          Yes
12          Yes
13          Yes
14          Yes
15          Yes
16          Yes
17          Yes
18          Yes
19          Yes
20          Yes
21          Yes
22          Yes
23          Yes
24          Yes
25          Yes
26          Yes
27          Yes
28          Yes
29          Yes
30          Yes
31          Yes
32          Yes
33          Yes
34          Yes
35          Yes
36          Yes
37          Yes
38          Yes
39          Yes
40          Yes
41          Yes
42          Yes
43          Yes
44          Yes
45          Yes
46          Yes
47          Yes
48          Yes
49          Yes
50          Yes
51          Yes
52          Yes
53          Yes
54          Yes
55          Yes
56          Yes


# **The Decision Tree**

In [ ]:
def compute_entropy(sample):
    ## :PARAM sample: The data sample of a discrete distribution
    
    ## If the sample is of size 1 or less, the sample is too small
    if len(sample) < 2:
        return 0

    ## Create an array for each unique value's proportion in the sample
    ## "normalise=True": creates a proportion of the unique counts on a scale of 0-1
    freq = np.array( sample.value_counts(normalize=True))

    ## Formula to calculate the entropy of the sample
    ## NOTE 1e-6 prevents the log from causing an error
    return -(freq * np.log2(freq + 1e-6)).sum()

In [ ]:
def compute_info_gain(samples, key, target):
  ### Infomration Gain: E(Y) - E(Y|X)

    ## Get each proportion of unique values for the key attribute
    values = samples[key].value_counts(normalize=True)
    weighted_average_entropy = 0

    for value, frequency in values.iteritems():
        index = samples[key]==value
        ## "sub_entropy": the entropy of one of the unique values of the sample
        sub_entropy = compute_entropy(target[index])
        ## "weighted_average_entropy": the summation of the weighted entropies
        weighted_average_entropy += frequency * sub_entropy
    
    ## "target_entropy": the target attribute's total entropy
    target_entropy = compute_entropy(target)

    ## Return the information gain through the subtraction of entropies
    return target_entropy - weighted_average_entropy

In [ ]:
class TreeNode:
    """
    A recursively defined data structure to store a tree.
    Each node can contain other nodes as its children
    """
    def __init__(self):
        ## Holds the sub nodes for recursion
        # Recursive structure, those elements of the same type (TreeNode)
        self.children = {}

        ## Starting out the decision is undecided
        self.decision = None 
        ## Splitting feature
        self.split_attribute = None

    def fit(self, samples, target):
        """
        The function accepts a training dataset, from which it builds the tree 
        structure to make decisions or to make children nodes (tree branches) 
        to do further inquiries
        :param samples: [n * p] n observed data samples of p attributes
        :param target: [n] target values
        """
        ## If the sample is empty \
        ## the decision should be the node parent's decision
        if len(samples) == 0:
            self.decision = "Yes"
            return

        ## Otherwise, calculate for the maximum information gain
        else:
            ## If there is only one unique value in the target column \
            ## the decision should be set to that unique value
            if len(target.unique()) == 1:
                self.decision = target.unique()[0]
                return
            else:    
                max_info_gain = 0
                ## Loop through the sample columns
                for key in samples.keys():
                    ## Call the function to compute the information gain \
                    ## for the current key
                    key_info_gain = compute_info_gain(samples, key, target)

                    ## If the current key's information gain is greater \
                    ## than the maximum info gain, store the greatest info gain among the keys
                    if key_info_gain > max_info_gain:
                        max_info_gain = key_info_gain
                        ## The split attribute will be set to the key \
                        ## with the greatest information gain
                        self.split_attribute = key

                print(f"Split by {self.split_attribute}, IG: {max_info_gain:.2f}")

                ## Create a dictionary for each unique children value
                ## This dictionary will be used to refer each value to a specific branch of the tree
                self.children = {}
                for value in samples[self.split_attribute].unique():
                    index = samples[self.split_attribute] == value
                    ## Recursion, create and store new children nodes from split key
                    self.children[value] = TreeNode()
                    self.children[value].fit(samples[index], target[index])


    def new_print(self, prefix=''):
        if self.split_attribute is not None:
            for k, v in self.children.items():
                v.new_print(f"{prefix}:When {self.split_attribute} is {k}")
        else:
            print(f"{prefix}:{self.decision}")

    def predict(self, sample):
        if self.decision is not None:
            # uncomment to get log information of code execution
            # print("Decision:", self.decision)
            return self.decision
        else: 
            # this node is an internal one, further queries about an attribute 
            # of the data is needed.
            attr_val = sample[self.split_attribute]
            child = self.children[attr_val]
            return child.predict(sample)

In [ ]:
# Build the decision tree
tree = TreeNode()

# Assign 
sample_training_data = X_train_processed[attributes]
target_training_data = y_train_processed[target]

tree.fit(sample_training_data, target_training_data)


Split by ('New cases',), IG: 0.02
Split by ('Active',), IG: 0.10
Split by ('New recovered',), IG: 0.32
Split by ('Confirmed',), IG: 1.00


In [ ]:
# Predict with testing dataset
sample_testing_data = X_test_processed[attributes]
target_testing_data = y_test_processed[target]

for i in range(len(y_test_processed.columns)):
    print(f"Test predict sample [{sample_testing_data.columns[i][0]}]: \n{sample_testing_data.iloc[i]}\n\tTarget: {y_test_processed.iloc[i]}")
    print(f"Making decision ...")

    pred = tree.predict(sample_testing_data.iloc[i])

Test predict sample [Confirmed]: 
Confirmed        0.0
Deaths           0.0
Recovered        0.0
Active           0.0
New cases        0.0
New deaths       0.0
New recovered    0.0
Name: 0, dtype: float64
	Target: Habitability    Yes
Name: 0, dtype: object
Making decision ...
Decision: Yes
